In [1]:
import pandas as pd
import mysql.connector
from db.db import parameter

In [25]:
db_connection = mysql.connector.connect(**parameter, ssl_disabled=True)

In [3]:
def load_sql_file(file) -> str:
    with open(file, "r") as fin:
        return fin.read()

In [4]:
query = load_sql_file("/home/work/mysql_server/command/mysql.sql")

In [5]:
df = pd.read_sql(query, db_connection)

/tmp/ipykernel_912512/2931324547.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, db_connection)


In [9]:
df["combination_count"] = 1

In [10]:
df.head()

,killchain,tactic,technique,collection_layer,datasource,url,combination_count
0,Delivery,Initial Access,Trusted Relationship,Host,Application Log: Application Log Content,https://attack.mitre.org/techniques/T1199,1
1,Delivery,Initial Access,Hardware Additions,Host,Application Log: Application Log Content,https://attack.mitre.org/techniques/T1200,1
2,Delivery,Initial Access,Hardware Additions,Host,Drive: Drive Creation,https://attack.mitre.org/techniques/T1200,1
3,Delivery,Initial Access,Hardware Additions,Host,Network Traffic: Network Traffic Flow,https://attack.mitre.org/techniques/T1200,1
4,Delivery,Initial Access,Hardware Additions,Network,Network Traffic: Network Traffic Flow,https://attack.mitre.org/techniques/T1200,1


In [15]:
pivot_table = df.pivot_table(
        index=["tactic", 'technique'],
        columns=['collection_layer', 'datasource'],
        values='combination_count',
        aggfunc='max',
        fill_value=0
)

In [21]:
pivot_table.shape

(170, 70)

In [23]:
pivot_table.reset_index()["technique"].nunique()

139

In [12]:
pivot_table.to_excel("to_excel2.xlsx")

In [26]:
query3 = load_sql_file("/home/work/mysql_server/command/mysql3.sql")
df = pd.read_sql(query3, db_connection)

/tmp/ipykernel_912512/2448752134.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query3, db_connection)


In [40]:
query3 = load_sql_file("/home/work/mysql_server/command/tech_url_dict.sql")
technique_url_dic = pd.read_sql(query3, db_connection).set_index("technique")["url"].to_dict()
query4 = load_sql_file("/home/work/mysql_server/command/tac_order_dict.sql")
tac_order_dic = pd.read_sql(query3, db_connection).set_index("technique")["url"].to_dict()

/tmp/ipykernel_912512/567180006.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  technique_url_dic = pd.read_sql(query3, db_connection).set_index("technique")["url"].to_dict()


In [30]:
dic = df.set_index("technique")["url"].to_dict()

In [32]:
pivot_table = pivot_table.reset_index()

In [33]:
pivot_table["url"] = pivot_table["technique"].map(dic)
pivot_table["url"] = pivot_table["technique"].map(dic)

In [35]:
pivot_table.set_index(["tactic", "technique"])

collection_layer                                                                      Host  \
datasource                           Active Directory: Active Directory Credential Request   
tactic     technique                                                                         
Collection Adversary-in-the-Middle                                                    0      
           Archive Collected Data                                                     0      
           Audio Capture                                                              0      
           Automated Collection                                                       0      
           Browser Session Hijacking                                                  0      

collection_layer                                                                       \
datasource                           Active Directory: Active Directory Object Access   
tactic     technique                                                                    
Collection Adversary-in-the-Middle                                                  0   
           Archive Collected Data                                                   0   
           Audio Capture                                                            0   
           Automated Collection                                                     0   
           Browser Session Hijacking                                                0   

collection_layer                                                                         \
datasource                           Active Directory: Active Directory Object Creation   
tactic     technique                                                                      
Collection Adversary-in-the-Middle                                                    0   
           Archive Collected Data                                                     0   
           Audio Capture                                                              0   
           Automated Collection                                                       0   
           Browser Session Hijacking                                                  0   

collection_layer                                                                         \
datasource                           Active Directory: Active Directory Object Deletion   
tactic     technique                                                                      
Collection Adversary-in-the-Middle                                                    0   
           Archive Collected Data                                                     0   
           Audio Capture                                                              0   
           Automated Collection                                                       0   
           Browser Session Hijacking                                                  0   

collection_layer                                                                             \
datasource                           Active Directory: Active Directory Object Modification   
tactic     technique                                                                          
Collection Adversary-in-the-Middle                                                    0       
           Archive Collected Data                                                     0       
           Audio Capture                                                              0       
           Automated Collection                                                       0       
           Browser Session Hijacking                                                  0       

collection_layer                                                               \
datasource                           Application Log: Application Log Content   
tactic     technique                                                            
Collection Adversary-in-the-Middle                                      

In [36]:
pivot_table.to_excel("sample.xlsx")